<a href="https://colab.research.google.com/github/jantuitman/deeplearning/blob/main/Training_met_simple_T5_om_vragen_te_maken.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers;
!pip install torch;
!pip install sklearn;
!pip install simplet5

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 53.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 96.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 27.8 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for sklearn: filename=sklearn-0.0.post1-py3-none-any.whl size=2344 sha256=afd882a60c246283a6ccf68be9bf11555564d99e06dea8f381d20674f09fd188
  Stored in directory: /root/.cache/pip/wheels/14/25/f7/1cc0956978ae479e75140219088deb7a36f60459df242b1a72
Successfully built sklearn
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ...

In [2]:
from transformers import  T5ForConditionalGeneration, Adafactor, T5Tokenizer, AutoTokenizer, PreTrainedModel
from simplet5 import SimpleT5
import torch
import time

import json

INFO:pytorch_lightning.utilities.seed:Global seed set to 42


In [3]:
# read a json file containing an array of objects
# each object has a "question" and "answer" field
# the question is the input and the answer is the output
# we will return a list of tuples
def read_dataset(filename):
    with open(filename) as f:
        data = json.load(f)
        # data has the following structure:
        # data = {
        #     "paragraphs": [
        #         {
        #             "translated_context": "text",
        #             "translated_question": "text",
        #             "translated_answers": [
        #                 "text",
        #                 "text",
        #                 "text"
        #             ],
        #             "is_impossible": true
        #         },
        #         ...
        #     ]
        # }
        # if is_impossible is true, then translated_answers is empty and we want to return the answer "Onbekend"
        # if is_impossible is false, then translated_answers is not empty and we want to return the first answer
        # we will return a list of tuples
        # we need to make a question_text from the translated_context and translated_question
        result= []
        for item in data:
            questions = '\n'.join(item['questions'])
            text = f"Maak vragen over deze tekst:\n{item['context']}"
            result.append((text, questions))
        return result

In [4]:


model_name='yhavinga/t5-base-dutch'
model = SimpleT5()
model.from_pretrained("t5",model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)




Downloading:   0%|          | 0.00/1.85k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/0.98M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.74k [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Downloading:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/850M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [7]:
from sklearn.model_selection import train_test_split
data = read_dataset('drive/MyDrive/OutputModels/questions.json')
train_data, val_data = train_test_split(data,test_size=0.1)

# Prepare the train dataset
#train_dataset = prepare_dataset(train_data,tokenizer)
#val_dataset = prepare_dataset(val_data,tokenizer)


In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
train_data[0]




('Maak vragen over deze tekst:\nIn 2012 vroegen privéscholen van goede kwaliteit in de Verenigde Staten een aanzienlijk collegegeld, bijna 40.000 dollar per jaar voor dagscholen in New York City, en bijna 50.000 dollar voor kostscholen. Het schoolgeld dekte echter niet de exploitatiekosten, vooral niet in internaten. De belangrijkste scholen, zoals de Groton School, hadden aanzienlijke schenkingen die opliepen tot honderden miljoenen dollars, aangevuld met fondsenwerving. Internaten met een reputatie van kwaliteit in de Verenigde Staten hebben een leerlingenpopulatie uit het hele land en zelfs de hele wereld en een kandidatenlijst die hun capaciteit ver overschrijdt.',
 'In welk jaar vroegen privéscholen van goede kwaliteit in de Verenigde Staten collegegeld?\nHoeveel dollar was het collegegeld per jaar voor dagscholen in New York City?\nDekte het collegegeld de exploitatiekosten?\nWelke scholen hadden aanzienlijke schenkingen?\nHeeft een internaat met een kwaliteitsreputatie in de Ver

In [13]:
import pandas as pd

train_df = pd.DataFrame(train_data,None,["source_text","target_text"])
eval_df = pd.DataFrame(val_data,None,["source_text","target_text"])

eval_df

,source_text,target_text
0,Maak vragen over deze tekst:\nDe vroegste gere...,Wanneer vonden de eerste geregistreerde incide...
1,Maak vragen over deze tekst:\nTot de bekendste...,Wat zijn experimenten in de structurele geolog...
2,Maak vragen over deze tekst:\nTerwijl de meest...,Verkopen alle internetapotheken geneesmiddelen...
3,"Maak vragen over deze tekst:\nFrederik Willem,...",Wie nodigde hugenoten uit om zich te vestigen?...
4,Maak vragen over deze tekst:\nHet Parlement ve...,Wanneer vergadert het Parlement gewoonlijk?\nH...
...,...,...
278,Maak vragen over deze tekst:\nVrouwen bleven g...,Wat was de relatie tussen Harvard en Radcliffe...
279,Maak vragen over deze tekst:\nIqbal vreesde ni...,Wat vreesde Iqbal?\nWaar bezocht Iqbal tijdens...
280,Maak vragen over deze tekst:\nOp 3 mei 1901 we...,Op welke datum gebeurde de Grote Brand van 190...
281,"Maak vragen over deze tekst:\nHet vierjarige, ...","Wat is het vierjarige, voltijdse undergraduate..."


In [14]:
# train
model.train(train_df=train_df, # pandas dataframe with 2 columns: source_text & target_text
            eval_df=eval_df, # pandas dataframe with 2 columns: source_text & target_text
            source_max_token_len = 512, 
            target_max_token_len = 128,
            batch_size = 8,
            max_epochs = 5,
            use_gpu = True,
            outputdir = "drive/MyDrive/OutputModels/simpleT5",
            early_stopping_patience_epochs = 0,
            precision = 32
            )

INFO:pytorch_lightning.utilities.distributed:GPU available: True, used: True
INFO:pytorch_lightning.utilities.distributed:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.distributed:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.accelerators.gpu:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type                       | Params
-----------------------------------------------------
0 | model | T5ForConditionalGeneration | 222 M 
-----------------------------------------------------
222 M     Trainable params
0         Non-trainable params
222 M     Total params
891.537   Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

/usr/local/lib/python3.8/dist-packages/pytorch_lightning/trainer/data_loading.py:132: UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
INFO:pytorch_lightning.utilities.seed:Global seed set to 42
/usr/local/lib/python3.8/dist-packages/pytorch_lightning/trainer/data_loading.py:132: UserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

In [16]:
context = "Willem Alexander is de koning van Nederland. Zijn ouders zijn koningin Beatrix en prins Claus. Hij is getrouwd met Maxima. Ze hebben drie kinderen, Alexia, Amalia en Anna."

input_text = f'Maak vragen over deze tekst:{context}\n'
model.load_model("t5","drive/MyDrive/OutputModels/simpleT5/simplet5-epoch-3", use_gpu=False)

print(model.predict(input_text))

['Wat is de koning van Nederland? Wie is de vader van Willem Alexander? Met wie heeft Willem Alexander drie kinderen? Hoeveel kinderen heeft Willem Alexander?']


In [21]:
def test(text):
  input_text = f'Maak vragen over deze tekst:{text}\n'
  print(model.predict(input_text))

In [23]:
test("""Nederland is een land in het noordwesten van Europa. 
Nederland heeft ongeveer 18 miljoen inwoners. 
De hoofdstad van Nederland is Amsterdam. Andere steden in Nederland zijn Rotterdam, Utrecht en Groningen.
De koning van Nederland heet Willem Alexander.
""")


['Wat is een land in het noordwesten van Europa? Hoeveel inwoners heeft Nederland? Wat is de hoofdstad van Nederland? Wat is de koning van Nederland? Wie heet de koning van Nederland?']


In [24]:
test("""Een timmerman maakt werkstukken van hout. 
Een timmerman moet goed om kunnen gaan met zaag en hamer. 
Een timmerman heeft een mbo-opleiding.
Timmermannen werken vaak in de bouw of in de meubelmakerij.
""")

['Wat maakt een timmerman van hout? Wat moet een timmerman kunnen omgaan met zaag en hamer? Wat is een mbo-opleiding voor timmerman? Waar werken timmermannen vaak? In welke branches werken timmermannen?']
